## Simulating melts of polymer chains

In [6]:
import warnings
warnings.filterwarnings('ignore')
import hoomd
import gsd
import gsd.hoomd
from hoomd.md import nlist

import unyt as u
import glob
import mbuild as mb
import matplotlib.pyplot as plt
import numpy as np
import os

import flowermd
from flowermd.utils import get_target_box_mass_density
from flowermd.base import Simulation, Molecule
from flowermd.library import FF_from_file
from flowermd.base.system import Pack

from polymer_dictionary import polymer_dictionary

# Rerun with the original seed first

In [7]:
# Updates atom names to fit with hoomd conventions
def espaloma_mol(file_path):
     mol = mb.load(file_path)
     for p in mol.particles():
           p.name = f"_{p.name}"
     return mol

## Single molecule sim:

In [8]:
key_list = sorted(list(polymer_dictionary.keys())) # Aligning dictionary and path to aid automation
path = os.getcwd() # Non-user specific path
molecule_list = sorted(glob.glob(path+"/mol2/*.mol2"))
molecule_list.pop(10) # Removing polymer not tested
print(key_list)

['PCPDTFBT_C11_BO', 'PCPDTFBT_C1_BO', 'PCPDTFBT_C3_BO', 'PCPDTFBT_C4_BO', 'PCPDTFBT_C5_BO', 'PCPDTPT_HD', 'PCPDTPT_ODD', 'PCPDTPT_eneODD', 'PCPDTPT_nC16', 'PCPDT_PT_eneHD', 'PIDTBT_nC16', 'PIDTCPDT_C11BO', 'PIDTFBT_C11_BO']


In [13]:
mol2_path = os.getcwd() + "/mol2/"
xml_path = os.getcwd() + "/xml/"

In [14]:
system_file = mol2_path + "/test_mol2/PCPDTFBT_C4_BO_50mer.mol2"
ff_filepath = xml_path + "PCPDTFBT_C4_BO.xml"

espmol = espaloma_mol(system_file)
molecule = Molecule(num_mols=1, compound=espmol)

molff = FF_from_file(ff_filepath)
system = Pack(molecules=molecule,density=0.01 * u.g/u.cm**3,packing_expand_factor=5)
system.apply_forcefield(r_cut=2.5, force_field=molff, auto_scale=True,remove_charges=True, remove_hydrogens=True)

KeyboardInterrupt: 

In [5]:
system.hoomd_snapshot
hoomd_forces = system.hoomd_forcefield
hoomd_forces
lj_force = hoomd_forces[3]
cpu = hoomd.device.CPU()
sim = Simulation.from_system(system=system, gsd_write_freq=1, log_write_freq=1, device=cpu, gsd_file_name="alternate_PCPDTPT_nC16_melt.gsd",log_file_name="alternate_PCPDTPT_nC16_melt.txt")

Initializing simulation state from a gsd.hoomd.Frame.


In [6]:
sim.run_NVT(n_steps=1, kT=5.0, tau_kt=1.0)
sim.flush_writers()

In [24]:
sim.reference_values

{'energy': unyt_quantity(1.046, 'kJ/mol'),
 'length': unyt_quantity(0.35635949, 'nm'),
 'mass': unyt_quantity(32.06, 'amu')}

In [4]:
tau = np.sqrt((32.06*0.35635949**2)/1.046)

In [5]:
print(tau)

1.9728962415951077


## Running sims of every polymer in the Danielsen et al. paper

In [5]:
pack_seeds = [600,700,800,900,1000,1100,1200,1300,1400,1500]

In [6]:
for pack_seed in pack_seeds:
    for i in range(len(key_list)):
        system_file = mol2_path + "10_mers/" + key_list[i] + "_10mer.mol2"
        ff_filepath = xml_path + key_list[i] + ".xml"
        gsd_path = os.getcwd() + "/gsd_files/10_mers/"+str(pack_seed)+"/"
        num_mers = "_10mer" # The number of monomers being simulated is determined by the mol2, this is for naming the resulting gsd and .txt files
        espmol = espaloma_mol(system_file)
        molecule = Molecule(num_mols=1, compound=espmol)
        cpu = hoomd.device.CPU()
        
        molff = FF_from_file(ff_filepath)
        system = Pack(molecules=molecule,density=0.01 * u.g/u.cm**3,seed=pack_seed, packing_expand_factor=5)
        system.apply_forcefield(r_cut=2.5, force_field=molff, auto_scale=True,remove_charges=True, remove_hydrogens=True)
        system.hoomd_snapshot
        hoomd_forces = system.hoomd_forcefield
        hoomd_forces
        lj_force = hoomd_forces[3]
        
        sim = Simulation.from_system(system=system, gsd_write_freq=5000, log_write_freq=5000, device=cpu, gsd_file_name=gsd_path+key_list[i]+num_mers+"_seed_"+str(pack_seed)+".gsd",log_file_name=gsd_path+key_list[i]+num_mers+"_seed_"+str(pack_seed)+".txt")
        sim.run_NVT(n_steps=5e5, kT=5.0, tau_kt=1.0)
        sim.flush_writers()

No charged group detected, skipping electrostatics.
Initializing simulation state from a gsd.hoomd.Frame.
Step 5000 of 500000; TPS: 473.52; ETA: 17.4 minutes
Step 10000 of 500000; TPS: 533.94; ETA: 15.3 minutes
Step 15000 of 500000; TPS: 569.05; ETA: 14.2 minutes
Step 20000 of 500000; TPS: 560.21; ETA: 14.3 minutes
Step 25000 of 500000; TPS: 559.39; ETA: 14.2 minutes
Step 30000 of 500000; TPS: 568.81; ETA: 13.8 minutes
Step 35000 of 500000; TPS: 570.93; ETA: 13.6 minutes
Step 40000 of 500000; TPS: 560.63; ETA: 13.7 minutes
Step 45000 of 500000; TPS: 560.98; ETA: 13.5 minutes
Step 50000 of 500000; TPS: 561.32; ETA: 13.4 minutes
Step 55000 of 500000; TPS: 560.97; ETA: 13.2 minutes
Step 60000 of 500000; TPS: 552.79; ETA: 13.3 minutes
Step 65000 of 500000; TPS: 556.16; ETA: 13.0 minutes
Step 70000 of 500000; TPS: 554.09; ETA: 12.9 minutes
Step 75000 of 500000; TPS: 551.02; ETA: 12.9 minutes
Step 80000 of 500000; TPS: 551.14; ETA: 12.7 minutes
Step 85000 of 500000; TPS: 551.16; ETA: 12.5 mi

IOStream.flush timed out


Step 225000 of 500000; TPS: 1109.24; ETA: 4.1 minutes
Step 230000 of 500000; TPS: 1110.48; ETA: 4.1 minutes
Step 235000 of 500000; TPS: 1111.65; ETA: 4.0 minutes
Step 240000 of 500000; TPS: 1113.23; ETA: 3.9 minutes
Step 245000 of 500000; TPS: 1114.15; ETA: 3.8 minutes
Step 250000 of 500000; TPS: 1107.29; ETA: 3.8 minutes
Step 255000 of 500000; TPS: 1108.47; ETA: 3.7 minutes
Step 260000 of 500000; TPS: 1109.65; ETA: 3.6 minutes
Step 265000 of 500000; TPS: 1111.08; ETA: 3.5 minutes
Step 270000 of 500000; TPS: 1112.1; ETA: 3.4 minutes
Step 275000 of 500000; TPS: 1112.42; ETA: 3.4 minutes
Step 280000 of 500000; TPS: 1112.61; ETA: 3.3 minutes
Step 285000 of 500000; TPS: 1112.99; ETA: 3.2 minutes
Step 290000 of 500000; TPS: 1106.21; ETA: 3.2 minutes
Step 295000 of 500000; TPS: 1106.19; ETA: 3.1 minutes
Step 300000 of 500000; TPS: 1105.62; ETA: 3.0 minutes
Step 305000 of 500000; TPS: 1105.99; ETA: 2.9 minutes
Step 310000 of 500000; TPS: 1106.25; ETA: 2.9 minutes
Step 315000 of 500000; TPS: 1


KeyboardInterrupt

